# Understanding Partitions, `glom`, and Narrow vs Wide Transformations

This notebook is designed for **students** learning PySpark on Databricks.

We will use:
- Synthetic data created with `spark.range()` (always available)
- A small sample from `samples.nyctaxi.trips` (built-in dataset)

Concepts covered:
1. What is a partition?
2. How to see the number of partitions
3. How to inspect partition contents with `glom`
4. Narrow vs Wide transformations
5. `repartition` vs `coalesce`


In [ ]:
from pyspark.sql import functions as F

# A tiny DataFrame with numbers 0..19
df = spark.range(0, 20).toDF("id")

print("Default partitions:", df.rdd.getNumPartitions())
display(df)


## 1. What is a partition?

- A **partition** is a chunk of your data stored and processed together.
- Spark parallelizes work **per partition**.
- More partitions = more parallelism (to a point).

Let's inspect how many partitions we have and what data is inside each partition using **`glom()`**.


In [ ]:
# `glom()` converts each partition into a list so you can see its contents
rdd = df.rdd
print("Number of partitions:", rdd.getNumPartitions())

partitioned = rdd.glom().collect()

for idx, part in enumerate(partitioned):
    print(f"Partition {idx}: {part}")


## 2. Controlling Number of Partitions

We can use:
- `repartition(n)` to **increase or decrease** partitions (full shuffle)
- `coalesce(n)` to **only decrease** partitions (avoid full shuffle when possible)


In [ ]:
# Repartition to 4 partitions (this causes a shuffle)
df4 = df.repartition(4)
print("df4 partitions:", df4.rdd.getNumPartitions())
for idx, part in enumerate(df4.rdd.glom().collect()):
    print(f"df4 - Partition {idx}: {part}")


In [ ]:
# Coalesce down to 2 partitions (tries to avoid full shuffle)
df2 = df4.coalesce(2)
print("df2 partitions:", df2.rdd.getNumPartitions())
for idx, part in enumerate(df2.rdd.glom().collect()):
    print(f"df2 - Partition {idx}: {part}")


## 3. Narrow vs Wide Transformations

### Narrow Transformations
- Each input partition contributes to **only one** output partition.
- Examples: `map`, `filter`, `withColumn`, simple `select`.
- No shuffle needed.

### Wide Transformations
- Input data may be **rearranged across partitions**.
- Examples: `groupBy`, `reduceByKey`, `distinct`, `join`, `repartition`.
- Causes a **shuffle** (data movement between nodes).


In [ ]:
# Narrow transformations example
narrow_df = (
    df
    .withColumn("id_times_2", F.col("id") * 2)   # withColumn = narrow
    .filter(F.col("id") % 2 == 0)                # filter = narrow
)

print("Partitions after narrow transformations:", narrow_df.rdd.getNumPartitions())
for idx, part in enumerate(narrow_df.rdd.glom().collect()):
    print(f"NARROW - Partition {idx}: {part}")


In [ ]:
# Wide transformation example: groupBy (causes shuffle)
grouped_df = (
    df
    .withColumn("key", F.col("id") % 3)
    .groupBy("key")       # wide transformation
    .agg(F.collect_list("id").alias("ids"))
)

display(grouped_df)


Behind the scenes, `groupBy`:
- Needs all rows with the same `key` together.
- So Spark has to **shuffle** rows across the cluster to regroup them.
- This is what makes it a **wide** transformation.


## 4. Narrow vs Wide on a Real Dataset

Let's look at a real built-in dataset: `samples.nyctaxi.trips`.


In [ ]:
nyc_df = spark.read.table("samples.nyctaxi.trips")
print("NYC Taxi partitions:", nyc_df.rdd.getNumPartitions())
display(nyc_df.limit(5))


In [ ]:
# Narrow: adding a column and filter
nyc_narrow = (
    nyc_df
    .withColumn("trip_duration_min",
                (F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long")) / 60.0)
    .filter(F.col("trip_distance") > 1.0)
)

print("NYC narrow partitions:", nyc_narrow.rdd.getNumPartitions())


In [ ]:
# Wide: groupBy passenger_count
nyc_grouped = (
    nyc_narrow
    .groupBy("passenger_count")   # wide (shuffle)
    .agg(F.avg("trip_distance").alias("avg_distance"))
)

display(nyc_grouped)


## 5. Summary (Important for Students)

- Spark splits data into **partitions**.
- **Narrow transformations**: data stays in the same partition (no shuffle).
- **Wide transformations**: data is shuffled across partitions (expensive).
- Use `glom()` on the **RDD** to inspect partition contents.
- Use `repartition()` to shuffle and change partition count.
- Use `coalesce()` when reducing partitions and you want to **avoid** a big shuffle.
